In [ ]:
from fastermodels.hf import *
from fastai.vision.all import *
from fastai.callback.all import *
from fasterai.core.criteria import *
import torch_pruning as tp
from torch_pruning.pruner import function
from torch_pruning.pruner.algorithms.scheduler import linear_scheduler
import timm

import torch
import torch.nn as nn
import torch.nn.functional as F

from fasterbench.benchmark import *
from fasterai.prune.all import *

In [ ]:
hf_model_name = 'tutorial'
model_name = 'alexnet'

## Get a model

In [ ]:
model = globals()[model_name]()
model_size = get_model_size(model)

In [ ]:
def get_dls(size, bs):
    path = URLs.IMAGENETTE_160
    source = untar_data(path)
    blocks=(ImageBlock, CategoryBlock)
    tfms = [RandomResizedCrop(size, min_scale=0.35), FlipItem(0.5)]
    batch_tfms = [Normalize.from_stats(*imagenet_stats)]

    csv_file = 'noisy_imagenette.csv'
    inp = pd.read_csv(source/csv_file)
    dblock = DataBlock(blocks=blocks,
               splitter=ColSplitter(),
               get_x=ColReader('path', pref=source),
               get_y=ColReader(f'noisy_labels_0'),
               item_tfms=tfms,
               batch_tfms=batch_tfms)

    return dblock.dataloaders(inp, path=source, bs=bs)

In [ ]:
dls = get_dls(160, 32)
learn = Learner(dls, model, metrics = [accuracy])
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,2.190084,2.294868,0.154650,00:05
1,1.957930,1.917212,0.328153,00:05
2,1.689020,1.605206,0.442293,00:05
3,1.430839,1.361510,0.538344,00:05
4,1.343451,1.291558,0.572994,00:05


In [ ]:
loss, acc = learn.validate()

## Upload the model

In [ ]:
metadata = {
    "model_name": hf_model_name,
    "architecture": model_name,
    "framework": "PyTorch",
    "datasets": ["ImageNet"],
    "task": "Classification",
    "metrics": {"Model Accuracy": acc},
    "usage_example": """""",
    "description": "A simple torchvision model",
    "license": "mit",
    "tags": ["classification"],
}

In [ ]:
# Generate the README content
readme_content = generate_readme(metadata)

# Save the README to a file
with open("README.md", "w") as f:
    f.write(readme_content)

print("README.md has been generated.")

README.md has been generated.


In [ ]:
create_repository(f"Nathan12/{hf_model_name}")

save_and_upload_model(f"Nathan12/{hf_model_name}", model, f"{model_name}", readme_path="README.md")

alexnet:   0%|          | 0.00/244M [00:00<?, ?B/s]

## Load the model from HuggingFace

In [ ]:
from huggingface_hub import hf_hub_download
import torch

# Download the model from Hugging Face
model_file = hf_hub_download(
    repo_id=f"Nathan12/{hf_model_name}",
    filename=model_name
)

# Load the model (architecture + weights)
model = torch.load(model_file)

print("Model loaded successfully!")

alexnet:   0%|          | 0.00/244M [00:00<?, ?B/s]

/tmp/ipykernel_11566/2452638795.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_file)


Model loaded successfully!


In [ ]:
learn = Learner(dls, model, metrics = [accuracy])
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,1.561073,1.590430,0.476688,00:05
1,1.508280,1.513178,0.504204,00:05
2,1.322130,1.321613,0.576815,00:05
3,1.143002,1.087752,0.644331,00:05
4,1.034787,1.018889,0.674140,00:05
